**ePartogram-GPT using Zephyr 7B Beta and Pytorch NLP**

**System Setup**

In [1]:
#install required packages
%%capture
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain langchain-community langchain-core langchain-huggingface sentence-transformers gradio pypdf SpeechRecognition
!pip install haystack-ai transformers accelerate bitsandbytes sentence_transformers bitsandbytes

In [2]:
# import audio packages
%%capture
!apt-get update
!apt-get install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio
!pip install spacy
!python -m spacy download en_core_web_sm

In [3]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
# import dependencies
import os
import gradio as gr
from google.colab import drive
import csv

import chromadb
#from langchain.llms import HuggingFacePipeline

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [5]:
# Prompt Engineering Packages
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

In [6]:
# Speech Recognition and Chat History Packages
import speech_recognition as sr
import re
import csv
from datetime import datetime, timedelta
# from pocketsphinx import LiveSpeech

**Define and Train Model**

In [7]:
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language="en-US")
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from speech recognition service; {e}"

def process_audio(audio_file):
    if audio_file is not None:
        try:
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file) as source:
                audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            return text
        except Exception as e:
            return f"Error in speech recognition: {str(e)}"
    return "No audio input received"

In [8]:
# specify model huggingface mode name
model_name = "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [9]:
# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [10]:
!pip install -U bitsandbytes
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [11]:
# mount google drive and specify folder path
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML1'
!ls '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML1'

Mounted at /content/drive
'Copy of ANC5_LifeThreatComp.pdf'
'Copy of Clinical Rules Table_4.25.16.pdf'
'Copy of WHO_Labour Care Guide_English.pdf'
"Copy of WHO LCG User's Manual.pdf"
'Copy of (WHO)Managing_Complications_Pregnancy_Childbirth.pdf'


**Retrieval Augmented Generation (RAG)**

In [12]:
# load pdf files
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()
print(documents)

# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Change the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO

#embed document chunks
vectordb = Chroma.from_documents(documents = all_splits, embedding = embeddings, persist_directory="chroma_db") #TO DO

# specify the retriever
retriever = vectordb.as_retriever()

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML1/Copy of WHO_Labour Care Guide_English.pdf', 'page': 0}, page_content='WHO LABOUR CARE GUIDE\nASSESSMENT\nPLAN\nOxytocin (U/L, drops/min)\nMedicine\nIV fluids\nBaseline  \nFHR <110, ≥160\nFHR  \ndeceleration L\nAmniotic fluid M+++, B\nFetal position P, T\nCaput +++\nMoulding +++\nContractions \nper 10 min ≤2, >5\nDuration of \ncontractions <20, >60\nCervix  \n[Plot X]\n10\n9 ≥ 2h\n8 ≥ 2.5h\n7 ≥ 3h\n6 ≥ 5h\n5 ≥ 6h\nDescent  \n[Plot O]\n5\n4\n3\n2\n1\n0\nBABYWOMANLABOUR PROGRESSMEDICATIONSHARED DECISION-MAKING SUPPORTIVE CARE\nCompanion N\nPain relief N\nOral fluid N\nPosture SP\nPulse <60, ≥120\nSystolic BP <80, ≥140\nDiastolic BP  ≥90\nTemperature ºC <35.0,  \n≥ 37.5\nUrine P++, A++\nALERT\nRisk factorsRuptured membranes [Date                              Time                      ]\nAbbreviations:     Y – Yes, N – No, D – Declined, U – Unknown, SP – Supine, MO – Mobile, E – Early, L – Late, V – Variable

<ipython-input-12-861146f92fb9>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

model_name = "HuggingFaceH4/zephyr-7b-beta"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    top_k=5,
    num_return_sequences=1
)

# max_length = 2048

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

<ipython-input-13-bef78914abc9>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


**Prompt Engineering**

In [23]:
from langchain.prompts import PromptTemplate

prompt_template = """<|system|>
You are an AI assistant specialized in analyzing and interpreting medical documents. Provide accurate and helpful information based on the given context and question. Always prioritize patient safety and adhere to medical best practices in your responses.
</s>
<|user|>
Context: {context}
Question: {question}
</s>
<|assistant|>
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [15]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

<ipython-input-15-e79c405c38a6>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


**NLP Processing of Chat Input and AI Output**

In [16]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def extract_relevant_info(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Define the entities we want to extract
    entities = {
        'Companion': None,
        'Pain Relief': None,
        'Oral fluid': None,
        'Posture': None,
        'Baseline FHR': None,
        'FHR Decceleration': None,
        'Amniotic fluid': None,
        'Fetal position': None,
        'Caput': None,
        'Moulding': None,
        'Pulse': None,
        'Systolic BP': None,
        'Diastolic BP': None,
        'Temperature': None,
        'Urine': None,
        'Contractions per 10 min': None,
        'Duration of contractions': None,
        'Cervix': None,
        'Descent': None,
        'Medication': None,
        'Assessment': None,
        'Plan': None
    }

    for ent in doc.ents:
      if ent.label_ == "PERSON":
          entities['Companion'] = ent.text
      elif ent.label_ == "ORG" and "pain" in ent.text.lower():
          entities['Pain Relief'] = ent.text
      elif ent.label_ == "CARDINAL":
          if "contractions" in ent.sent.text.lower():
              entities['Contractions per 10 min'] = ent.text
          elif "seconds" in ent.sent.text.lower():
              entities['Duration of contractions'] = ent.text
          elif "pulse" in ent.sent.text.lower():
              entities['Pulse'] = ent.text
          elif "fhr" in ent.sent.text.lower() or "heart rate" in ent.sent.text.lower():
              entities['Baseline FHR'] = ent.text
          elif "cm" in ent.sent.text.lower():
              entities['Cervix'] = ent.text
      elif ent.label_ == "DATE":
          if "weeks" in ent.text.lower():
              entities['Gestational Age'] = ent.text

    # Additional extraction for blood pressure
    for token in doc:
        if token.text == "/" and token.i > 0 and token.i < len(doc) - 1:
            prev_token = doc[token.i - 1]
            next_token = doc[token.i + 1]
            if prev_token.like_num and next_token.like_num:
                entities['Systolic BP'] = prev_token.text
                entities['Diastolic BP'] = next_token.text

    # Extraction for specific phrases
    if "no decelerations" in doc.text.lower():
        entities['FHR Decceleration'] = "No"
    if "membranes intact" in doc.text.lower():
        entities['Amniotic fluid'] = "Intact"
    if "head all up" in doc.text.lower():
        entities['Fetal position'] = "Head all up"
    if "mostly effaced" in doc.text.lower():
        entities['Cervix'] += " mostly effaced"

    return entities

In [ ]:
def nag_function(digital_LCG):
  # digital_LCG is a df with all the variables in the LCG (timestamp, etc..)
  # Create thresholds based on current directory


In [17]:
def save_chat_to_csv(query: str, filename: str = '/content/drive/MyDrive/digital_lcg.csv'):
    # Define parameters to LCG
    fields = ['Timestamp', 'Companion', 'Pain Relief', 'Oral fluid', 'Posture', 'Gestational Age', 'Baseline FHR',
              'FHR Decceleration', 'Amniotic fluid', 'Fetal position', 'Caput', 'Moulding',
              'Pulse', 'Systolic BP', 'Diastolic BP', 'Temperature', 'Urine',
              'Contractions per 10 min', 'Duration of contractions', 'Cervix', 'Descent',
              'Medication', 'Assessment', 'Plan']

    # Read existing data from digital lcg
    existing_data = []
    try:
        with open(filename, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            existing_data = list(reader)
    except FileNotFoundError:
        pass

    # Process new query
    info = extract_relevant_info(query)
    timestamp = datetime.now()

    # Case where there are multiple entries within a 30 minute span containing different parameters
    recent_entry = next((entry for entry in reversed(existing_data)
                         if datetime.strptime(entry['Timestamp'], '%Y-%m-%d %H:%M:%S') > timestamp - timedelta(minutes=30)),
                        None)

    if recent_entry:
        # Update the recent entry
        for key, value in info.items():
            if value is not None:
                recent_entry[key] = value
    else:
        # Create a new entry
        new_entry = {field: '' for field in fields}
        new_entry['Timestamp'] = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        new_entry.update(info)
        existing_data.append(new_entry)

    # Write all data back to the CSV
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fields)
        writer.writeheader()
        writer.writerows(existing_data)

**UI Development**

In [18]:
def get_response(query: str, chat_history: list):
    result = qa_chain({"question": query, "chat_history": chat_history})
    return result["answer"]

In [25]:
def create_conversation(message, history):
    try:
        response = get_response(message, history)
        assistant_response = response.split("<|assistant|>")[-1].strip()
        formatted_response = f"Question: {message}\n\nHelpful Answer: {assistant_response}"
        history.append((message, formatted_response))
        save_to_csv_message = message

        # Segment the message into the WHO Labor Care Guide
        save_chat_to_csv(save_to_csv_message)
        return "", history
    except Exception as e:
        error_message = f"Question: {message}\n\nHelpful Answer: An error occurred: {str(e)}"
        history.append((message, error_message))
        return "", history

with gr.Blocks() as demo:
    gr.Markdown("ePartogram-GPT")

    chatbot = gr.Chatbot(label="Chat History")
    msg = gr.Textbox(label="Enter your question", placeholder="Type your question here...")
    audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Or speak your question")
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(lambda: "", inputs=[], outputs=[msg])
    audio_input.change(process_audio, inputs=[audio_input], outputs=[msg])

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9c74a9feb968c9380.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Test Cases**

**Test Query 1:** Mrs. A, Para 0+0 presents in labor at 39 weeks , uneventful ANC, Contractions 1/10 each 20 seconds, Pulse 88, BP 130/80, FHR 140/min, no decelerations, Head all up, Cervix mostly effaced, membranes intact, 4 cm dilatated. Assess the state of the patient.

In [21]:
# TEST QUERY 1
query = "Mrs. A, Para 0+0 presents in labor at 39 weeks , uneventful ANC, Contractions 1/10 each 20 seconds, Pulse 88, BP 130/80, FHR 140/min, no decelerations, Head all up, Cervix mostly effaced, membranes intact, 4 cm dilatated. Assess the state of the patient."
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
print(response)

<ipython-input-19-a2d99e93b188>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query, "chat_history": chat_history})
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|system|>
You are an AI assistant specialized in analyzing and interpreting medical documents. Provide accurate and helpful information based on the given context and question. Always prioritize patient safety and adhere to medical best practices in your responses.
</s>
<|user|>
Context: Unsatisfactory Progress of Labour S-77 
 
Figure S-9, page S-78, is a sample partograph showing arrest of dilatation 
and descent in the active phase of labour. Fetal distress and third degree 
moulding, together with arrest of dilatation and descent in the active phase 
of labour in the presence of adequate uterine contractions, indicate 
obstructed labour. 
• The woman was admitted in active labour at 10:00 with: 
- fetal head 3/5 palpable; 
- cervix dilated 4 cm; 
- three contractions in 10 minutes, each lasting 20–40 seconds; 
- clear amniotic fluid draining; 
- first degree moulding. 
• At 14:00: 
- fetal head still 3/5 palpable; 
- cervix dilated 6 cm and to the right of the alert line; 
- sligh

In [22]:
# Actual AI Result
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)

Based on the given medical documentation, it appears that Mrs. A is currently in labor at 39 weeks gestation with no significant antepartum complications. Her contractions are mild and irregular, occurring once every 20 seconds. Her vital signs are stable with a pulse rate of 88 and a blood pressure of 130/80. Her fetal heart rate is within normal range at 140 beats per minute and has not shown any decelerations. Her cervix is mostly effaced and dilated to 4 centimeters. The membranes are still intact. These findings indicate that Mrs. A is in the early stages of labor, and further monitoring and assessment will be necessary to evaluate the progress of the labor and determine the need for any interventions or medical management. It is important to continue monitoring the fetal heart rate and assessing the strength and frequency of contractions, as well as checking for any signs of fetal distress or abnormalities. Additionally, the cervical dilatation should be regularly evaluated to en

**Test Query 2:** Mrs. A, Para 0+0 presents in labor at 39 weeks , uneventful ANC, Contractions 1/10 each 20 seconds, Pulse 88, BP 130/80, FHR 140/min, no decelerations, Head all up, Cervix mostly effaced, membranes intact, 4 cm dilated. Provide a plan of care.

In [ ]:
# TEST QUERY 2:
query = "Mrs. A, Para 0+0 presents in labor at 39 weeks , uneventful ANC, Contractions 1/10 each 20 seconds, Pulse 88, BP 130/80, FHR 140/min, no decelerations, Head all up, Cervix mostly effaced, membranes intact, 4 cm dilated. Provide a plan of care."
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
print(response)

<|system|>
You are an AI assistant specialized in analyzing and interpreting medical documents. Provide accurate and helpful information based on the given context and question. Always prioritize patient safety and adhere to medical best practices in your responses.
</s>
<|user|>
Context: 11
HOW TO USE THE LABOUR CARE GUIDE
Example of how to complete Section 2
Date: June 07, 2020 Time 06:00
Mary Jane received a general and clinical assessment, and she has been admitted to the labour 
ward. 
She is monitored by the midwife on duty but she is not accompanied by a relative or someone 
from her social network. 
She reports feeling significant pain due to the uterine contractions, and requests pain relief.
She drank a fruit juice and is walking.
The midwife caring for and monitoring Mary Jane during labour offered her a companion of 
her choice. Mary Jane wanted to be accompanied by her sister. The midwife gave directions to 
Mary Jane’s sister as to when and how to call for assistance. 
Gi

In [ ]:
# Actual AI Result
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)

Based on the medical document and current state of Mrs. A's labor, the following plan of care is recommended:

1. Fetal Heart Rate Monitoring: Fetal heart rate monitoring should be initiated every 15-30 minutes, with documentation of any variations in the pattern. Prompt intervention, such as administering oxygen therapy, intravenous fluids, or uterine stimulants, should be initiated if any concerning changes in the fetal heart rate pattern are observed.

2. Cervical Dilation and Contractions Monitoring: Close monitoring of cervical dilation and contractions should be maintained. If the cervix does not dilate beyond 4 cm after eight hours of regular contractions, augmentation of labor with oxytocin may be necessary.

3. Pain Management: Adequate pain management should be provided, such as administering analgesia and anesthesia as required.

4. Hydration Status Monitoring: Close monitoring of hydration status should be initiated, and intravenous fluids may be necessary to manage any deh

**Test Query 3:** Mrs. C, 20 years old Para 0+1, TOP at 12 weeks 2 years ago, iron deficiency anemia on treatment with iron. Last hemoglobin 10.5g. Presents at 39 weeks with labor pains for 4 hours, Examination  revealed active labor, ARM done, clear amniotic fluid, normal progress, Buscopan given, instructed her sister to walk with Mrs. C. Comment on plan

In [23]:
query = "Mrs. C, 20 years old Para 0+1, TOP at 12 weeks 2 years ago, iron deficiency anemia on treatment with iron. Last hemoglobin 10.5g. Presents at 39 weeks with labor pains for 4 hours, Examination  revealed active labor, ARM done, clear amniotic fluid, normal progress, Buscopan given, instructed her sister to walk with Mrs. C. Comment on plan"
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
print(response)

<|system|>
You are an AI assistant specialized in analyzing and interpreting medical documents. Provide accurate and helpful information based on the given context and question. Always prioritize patient safety and adhere to medical best practices in your responses.
</s>
<|user|>
Context: 11
HOW TO USE THE LABOUR CARE GUIDE
Example of how to complete Section 2
Date: June 07, 2020 Time 06:00
Mary Jane received a general and clinical assessment, and she has been admitted to the labour 
ward. 
She is monitored by the midwife on duty but she is not accompanied by a relative or someone 
from her social network. 
She reports feeling significant pain due to the uterine contractions, and requests pain relief.
She drank a fruit juice and is walking.
The midwife caring for and monitoring Mary Jane during labour offered her a companion of 
her choice. Mary Jane wanted to be accompanied by her sister. The midwife gave directions to 
Mary Jane’s sister as to when and how to call for assistance. 
Gi

In [26]:
# Actual AI Result
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)

The use of Buscopan (hyoscine butylbromide) has been administered to Mrs. C to manage the intensity of her labor pains. This medication can help to relax the uterus and decrease the frequency and intensity of contractions, which may provide some relief to Mrs. C during labor. However, it's essential to consider other necessary measures to ensure safe delivery for both Mrs. C and her newborn.
As Mrs. C has a history of iron deficiency anemia, close monitoring of her hemoglobin levels and the management of any blood loss will be required during labor and childbirth. Iron deficiency anemia during pregnancy can lead to increased blood loss during delivery, which may exacerbate an already low hemoglobin level. Therefore, it's crucial to manage any blood loss effectively during delivery to prevent complications. The administration of iron and folate supplements, especially in the postpartum period, may also be necessary to replenish any iron and folate stores depleted during labor and childb

**Test Query 4:** (Bony Obstruction) A 28-year-old woman presents to the labor and delivery unit at 41 weeks gestation with regular contractions. After 18 hours of labor, she has made minimal cervical progress despite adequate uterine contractions. The fetus remains high in the pelvis, and the cervix is 6 cm dilated. Assess state of patient.

In [26]:
query = "A 28-year-old woman presents to the labor and delivery unit at 41 weeks gestation with regular contractions. After 18 hours of labor, she has made minimal cervical progress despite adequate uterine contractions. The fetus remains high in the pelvis, and the cervix is 6 cm dilated. Assess state of patient."
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
print(response)

<ipython-input-24-a2d99e93b188>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query, "chat_history": chat_history})
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|system|>
You are an AI assistant specialized in analyzing and interpreting medical documents. Provide accurate and helpful information based on the given context and question. Always prioritize patient safety and adhere to medical best practices in your responses.
</s>
<|user|>
Context: Unsatisfactory Progress of Labour S-77 
 
Figure S-9, page S-78, is a sample partograph showing arrest of dilatation 
and descent in the active phase of labour. Fetal distress and third degree 
moulding, together with arrest of dilatation and descent in the active phase 
of labour in the presence of adequate uterine contractions, indicate 
obstructed labour. 
• The woman was admitted in active labour at 10:00 with: 
- fetal head 3/5 palpable; 
- cervix dilated 4 cm; 
- three contractions in 10 minutes, each lasting 20–40 seconds; 
- clear amniotic fluid draining; 
- first degree moulding. 
• At 14:00: 
- fetal head still 3/5 palpable; 
- cervix dilated 6 cm and to the right of the alert line; 
- sligh

In [27]:
# Actual AI Result
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)

Based on the given medical context and question, the patient presents with unsatisfactory progress of labor, specifically a prolonged active phase and cervical dilatation to the right of the alert line on the partograph. The fetus remains high in the pelvis despite adequate uterine contractions. The patient's cervical progress has been minimal, despite 18 hours of labor at 41 weeks gestation. 

At this point, the patient may be experiencing emotional distress related to the prolonged labor and lack of progress. It is important to provide supportive care, such as offering a warm bath or shower, breathing techniques, and reassurance. Anxiety and fear should also be addressed through supportive care, which can include relaxation techniques, distraction methods, and the presence of a support person. 

In addition to emotional support, the patient should be closely monitored for signs of fetal distress, such as decreased fetal heart rate or meconium-stained amniotic fluid. Fetal heart rate 

Issue with Test Query 4: Oxytocin would actually make things worse, so we need to add specific RAG rules here

**Managing Newborn Problems Learning Resource Cases**

**Test Query 5:** You have completed your assessment of Baby A. and your main findings include the following: Baby A. was born at term and weighed 3.5 kg. Fetal distress was noted toward the end of labour and Baby A. did not breathe spontaneously at birth; she was resuscitated for four minutes using a bag and mask. Her respiratory rate is now consistently between 60 and 70 breaths per minute, with chest indrawing. There were no other abnormalities detected on examination. Baby A.’s temperature is 36.7°C. Her blood glucose is 85 mg/dl. There is no history of maternal uterine infection or fever anytime from the onset of labour to the present time, rupture of membranes for more than 18 hours before birth, or syphilis. Mrs. A.’s pregnancy was uneventful and, apart from a prolonged expulsive phase of labour and fetal distress, there were no other complications associated with the labour and birth. Based on these findings, what is Baby A.’s diagnosis (problem/need) and why?

In [28]:
query = "You have completed your assessment of Baby A. and your main findings include the following: Baby A. was born at term and weighed 3.5 kg. Fetal distress was noted toward the end of labour and Baby A. did not breathe spontaneously at birth; she was resuscitated for four minutes using a bag and mask. Her respiratory rate is now consistently between 60 and 70 breaths per minute, with chest indrawing. There were no other abnormalities detected on examination. Baby A.’s temperature is 36.7°C. Her blood glucose is 85 mg/dl. There is no history of maternal uterine infection or fever anytime from the onset of labour to the present time, rupture of membranes for more than 18 hours before birth, or syphilis. Mrs. A.’s pregnancy was uneventful and, apart from a prolonged expulsive phase of labour and fetal distress, there were no other complications associated with the labour and birth. Based on these findings, what is Baby A.’s diagnosis (problem/need) and why?"
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
response_filtered = response.split("<|assistant|>")[-1].strip()
print(f"""
{response_filtered}
""")


Baby A.'s diagnosis is respiratory distress, which was noted towards the end of labor and persists after birth. This diagnosis is based on Baby A.'s need for resuscitation using a bag and mask at birth and her consistently low respiratory rate between 60 and 70 breaths per minute with chest indrawing.

To manage Baby A.'s respiratory distress, initial steps may include close monitoring of her respiratory rate, oxygen saturation, and chest movements. Supplemental oxygen may be required to maintain adequate oxygenation levels. A nasogastric tube may be inserted for feeding, as sucking and swallowing can cause further exhaustion during the initial stages of respiratory distress. Additionally, chest physiotherapy may help to clear any secretions from the lungs, which can further exacerbate respiratory distress. The medical team may also consider administering surfactant, which can help to improve lung compliance and reduce the work of breathing. Close observation, regular assessment, and 

In [22]:
query = "hi"
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)

<ipython-input-18-a2d99e93b188>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query, "chat_history": chat_history})
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Greetings, the context provided is an introduction section of a medical document. Unfortunately, the message "hi" does not seem to be related to the medical content. If you can provide some additional context, such as the name of the medical document and the reason why you're contacting us, I may be able to help you further. Otherwise, you may want to check if there's a different document or context that you've accidentally shared. Thank you.


In [ ]:
query = ""
chat_history = []  # Initialize an empty chat history
response = get_response(query, chat_history)
response_filtered = response.split("<|assistant|>")[-1].strip()
print(response_filtered)